# Merging Climate Knowledge Portal Data

In [1]:
# For multiple output per cell
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
# DATASET_FOLDER = '/media/data-nvme/dev/datasets/WorldBank/'
DATASET_FOLDER = "../../datasets/precipitation/"

SPARK_MASTER = "spark://192.168.0.9:7077"
APP_NAME = "Merge PRCP GKP"
input_folder = DATASET_FOLDER
output = DATASET_FOLDER + "../wb_gkp_precipitation"

In [3]:
import os
import pandas as pd

# Merge projection on same line

In [4]:
import pandas as pd
from pyspark import SparkContext

# from pyspark import SparkConf
from pyspark.sql import SparkSession

# from pyspark.sql.window import Window
from pyspark.sql.types import FloatType
import pyspark.sql.functions as F

import shutil

## Connect to Spark

In [6]:
print("Create Spark session")
spark = SparkSession.builder.master(SPARK_MASTER).appName(APP_NAME).getOrCreate()
sc = spark.sparkContext

Create Spark session


In [7]:
#!ls /media/data-nvme/dev/datasets/WorldBank/

# Load

In [8]:
# df = spark.read.format('csv').option('header',True).option('multiLine', True).load(f'{DATASET_FOLDER}../projection_precipitation_clean_2020-12-01.csv')
df = spark.read.csv(
    f"{DATASET_FOLDER}../projection_precipitation_clean_2020-12-01.csv",
    inferSchema=True,
    header=True,
)

In [9]:
# df.columns

In [10]:
df.count()

6039651

In [12]:
df.printSchema()

root
 |-- 5-day_rain_sum_max_25_years_mm: double (nullable = true)
 |-- year: string (nullable = true)
 |-- model: string (nullable = true)
 |-- statistics: string (nullable = true)
 |-- ISO3: string (nullable = true)
 |-- projection_rcp: string (nullable = true)
 |-- daily_rain_max_25_years_mm: double (nullable = true)
 |-- nb_days_with_rain_>_50mm: double (nullable = true)
 |-- monthly_prcp_mm: double (nullable = true)
 |-- Country: string (nullable = true)
 |-- 5-day_rain_sum_max_10_years_mm: double (nullable = true)
 |-- largest_5-day_rain_sum_mm: double (nullable = true)
 |-- nb_days_with_rain_>_20mm: double (nullable = true)
 |-- rain_from_very_wet_days_percent: double (nullable = true)
 |-- daily_rain_max_10_years_mm: double (nullable = true)
 |-- largest_month_rain_25_years_mm: double (nullable = true)
 |-- largest_single_day_rain_mm: double (nullable = true)
 |-- largest_month_rain_10_years_mm: double (nullable = true)
 |-- month: double (nullable = true)



In [13]:
df.show(3)

+------------------------------+---------+--------------------+-----------+----+--------------+--------------------------+------------------------+---------------+-------+------------------------------+-------------------------+------------------------+-------------------------------+--------------------------+------------------------------+--------------------------+------------------------------+-----+
|5-day_rain_sum_max_25_years_mm|     year|               model| statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|nb_days_with_rain_>_50mm|monthly_prcp_mm|Country|5-day_rain_sum_max_10_years_mm|largest_5-day_rain_sum_mm|nb_days_with_rain_>_20mm|rain_from_very_wet_days_percent|daily_rain_max_10_years_mm|largest_month_rain_25_years_mm|largest_single_day_rain_mm|largest_month_rain_10_years_mm|month|
+------------------------------+---------+--------------------+-----------+----+--------------+--------------------------+------------------------+---------------+-------+-------------

In [24]:
yearly_features = [
    "daily_rain_max_25_years_mm",
    "daily_rain_max_10_years_mm",
    "5-day_rain_sum_max_10_years_mm",
    "5-day_rain_sum_max_25_years_mm",
    "largest_month_rain_10_years_mm",
    "largest_month_rain_25_years_mm",
]

monthly_features = [
    "monthly_prcp_mm",
    "largest_single_day_rain_mm",
    "rain_from_very_wet_days_percent",
    "largest_5-day_rain_sum_mm",
    "nb_days_with_rain_>_50mm",
    "nb_days_with_rain_>_20mm",
]

df.createOrReplaceTempView("future")
df_yearly = spark.sql(
    """
SELECT year, month, model, statistics, ISO3, projection_rcp, daily_rain_max_25_years_mm
    FROM future
    WHERE daily_rain_max_25_years_mm IS NOT NULL AND ISO3 IS NOT NULL AND year IS NOT NULL AND statistics IS NOT NULL
"""
)
df_yearly.count()

76359

In [25]:
df_yearly = None
for feature in yearly_features:
    if not df_yearly:
        df_yearly = spark.sql(
            "SELECT year, month, model, statistics, ISO3, projection_rcp, `"
            + feature
            + "` "
            + "FROM future"
            + " WHERE `"
            + feature
            + "` IS NOT NULL AND ISO3 IS NOT NULL AND year IS NOT NULL AND statistics IS NOT NULL"
        )
    else:
        sql = (
            "SELECT year, month, model, statistics, ISO3, projection_rcp, `"
            + feature
            + "`"
        )
        sql += " FROM future WHERE `" + feature + "` IS NOT NULL"
        df_temp = spark.sql(sql)
        # df_temp.show(3)
        df_temp.createOrReplaceTempView("temp")
        df_yearly.createOrReplaceTempView("yearly_table")
        sql = (
            "SELECT yearly_table.*, temp.`"
            + feature
            + "` FROM yearly_table LEFT JOIN temp"
            + " ON yearly_table.year=temp.year AND yearly_table.statistics=temp.statistics "
            + " AND yearly_table.model=temp.model AND yearly_table.ISO3=temp.ISO3 AND yearly_table.projection_rcp=temp.projection_rcp"
        )
        # print(sql)
        df_yearly = spark.sql(sql)
df_yearly.createOrReplaceTempView("yearly_table")

In [ ]:
df_monthly = None
for feature in monthly_features:
    if not df_monthly:
        df_monthly = spark.sql(
            "SELECT year, month, model, statistics, ISO3, projection_rcp, `"
            + feature
            + "` "
            + "FROM future"
            + " WHERE `"
            + feature
            + "` IS NOT NULL AND ISO3 IS NOT NULL AND year IS NOT NULL AND statistics IS NOT NULL AND month IS NOT NULL"
        )
        # df_monthly.show(3)
    else:
        sql = (
            "SELECT year, month, model, statistics, ISO3, projection_rcp, `"
            + feature
            + "`"
        )
        sql += " FROM future WHERE `" + feature + "` IS NOT NULL"
        df_temp = spark.sql(sql)
        # df_temp.show(3)
        df_temp.createOrReplaceTempView("temp")
        df_monthly.createOrReplaceTempView("monthly_table")
        sql = (
            "SELECT monthly_table.*, temp.`"
            + feature
            + "` FROM monthly_table LEFT JOIN temp"
            + " ON monthly_table.year=temp.year AND monthly_table.month=temp.month "
            + " AND monthly_table.model=temp.model AND monthly_table.ISO3=temp.ISO3 AND monthly_table.projection_rcp=temp.projection_rcp"
            + " AND monthly_table.month=temp.month"
        )
        # print(sql)
        df_monthly = spark.sql(sql)
df_monthly.createOrReplaceTempView("monthly_table")

## Save

In [39]:
df_yearly.write.csv(
    f"{DATASET_FOLDER}../projection_preciptation_yearly_merged-2020-12-02", header=True
)

In [40]:
df_monthly.write.csv(
    f"{DATASET_FOLDER}../projection_preciptation_monthly_merged-2020-12-02", header=True
)

In [41]:
import glob


def spark_merge_with_pandas(folder, outfilename):
    df_temp = pd.DataFrame()
    for filename in glob.glob(folder + "/*.csv"):
        df = pd.read_csv(filename)
        df_temp = df_temp.append(df)
    df_temp.to_csv(outfilename, index=False)
    del df_temp


spark_merge_with_pandas(
    f"{DATASET_FOLDER}../projection_preciptation_yearly_merged-2020-12-02",
    f"{DATASET_FOLDER}../projection_preciptation_yearly_merged-2020-12-02.csv",
)
spark_merge_with_pandas(
    f"{DATASET_FOLDER}../projection_preciptation_monthly_merged-2020-12-02",
    f"{DATASET_FOLDER}../projection_preciptation_monthly_merged-2020-12-02.csv",
)

## Check

In [26]:
spark.sql(
    "SELECT * FROM yearly_table WHERE ISO3='FRA' AND year='2020-2039' AND projection_rcp='rcp26' \
AND model='bcc_csm1_1' AND daily_rain_max_25_years_mm IS NOT NULL"
).show(10)

+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|     year|month|     model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|daily_rain_max_10_years_mm|5-day_rain_sum_max_10_years_mm|5-day_rain_sum_max_25_years_mm|largest_month_rain_10_years_mm|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|2020-2039| null|bcc_csm1_1|Annual Anomaly| FRA|         rcp26|                   1.51429|                   1.26548|                       2.44999|                       2.95582|                       -4.7716|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+--------

In [32]:
spark.sql(
    "SELECT * FROM monthly_table WHERE ISO3='FRA' AND year='2020-2039' AND projection_rcp='rcp26' \
AND model='bcc_csm1_1'"
).show(10)

+---------+-----+----------+-----------+----+--------------+---------------+--------------------------+-------------------------------+-------------------------+------------------------+------------------------+
|     year|month|     model| statistics|ISO3|projection_rcp|monthly_prcp_mm|largest_single_day_rain_mm|rain_from_very_wet_days_percent|largest_5-day_rain_sum_mm|nb_days_with_rain_>_50mm|nb_days_with_rain_>_20mm|
+---------+-----+----------+-----------+----+--------------+---------------+--------------------------+-------------------------------+-------------------------+------------------------+------------------------+
|2020-2039|  6.0|bcc_csm1_1|Jun Average| FRA|         rcp26|       41.56588|                   -0.8905|                         -0.557|                  -2.4796|                    null|                 0.00722|
|2020-2039|  9.0|bcc_csm1_1|Sep Average| FRA|         rcp26|      48.931923|                   0.26791|                         -0.743|                 

In [42]:
spark.sql(
    "SELECT * FROM yearly_table WHERE ISO3='FRA' AND year='2020-2039' AND projection_rcp='rcp26' \
AND model='bcc_csm1_1' AND daily_rain_max_25_years_mm IS NOT NULL"
).show(10)

+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|     year|month|     model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|daily_rain_max_10_years_mm|5-day_rain_sum_max_10_years_mm|5-day_rain_sum_max_25_years_mm|largest_month_rain_10_years_mm|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|2020-2039| null|bcc_csm1_1|Annual Anomaly| FRA|         rcp26|                   1.51429|                   1.26548|                       2.44999|                       2.95582|                       -4.7716|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+--------

In [44]:
spark.sql(
    "SELECT * FROM yearly_table WHERE ISO3='FRA' AND year='2020-2039' AND projection_rcp='rcp26' \
AND model='bcc_csm1_1' AND daily_rain_max_10_years_mm IS NOT NULL"
).show(10)

+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|     year|month|     model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|daily_rain_max_10_years_mm|5-day_rain_sum_max_10_years_mm|5-day_rain_sum_max_25_years_mm|largest_month_rain_10_years_mm|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|2020-2039| null|bcc_csm1_1|Annual Anomaly| FRA|         rcp26|                   1.51429|                   1.26548|                       2.44999|                       2.95582|                       -4.7716|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+--------

In [45]:
spark.sql(
    "SELECT * FROM future WHERE ISO3='FRA' AND year='2020-2039' AND projection_rcp='rcp26' AND model='bcc_csm1_1' AND daily_rain_max_25_years_mm IS NOT NULL"
).show(10)

+------------------------------+---------+----------+--------------+----+--------------+--------------------------+------------------------+---------------+-------+------------------------------+-------------------------+------------------------+-------------------------------+--------------------------+------------------------------+--------------------------+------------------------------+-----+
|5-day_rain_sum_max_25_years_mm|     year|     model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|nb_days_with_rain_>_50mm|monthly_prcp_mm|Country|5-day_rain_sum_max_10_years_mm|largest_5-day_rain_sum_mm|nb_days_with_rain_>_20mm|rain_from_very_wet_days_percent|daily_rain_max_10_years_mm|largest_month_rain_25_years_mm|largest_single_day_rain_mm|largest_month_rain_10_years_mm|month|
+------------------------------+---------+----------+--------------+----+--------------+--------------------------+------------------------+---------------+-------+------------------------------+---

In [47]:
spark.sql(
    "SELECT * FROM yearly_table WHERE ISO3='FRA' AND projection_rcp='rcp26' \
AND model='bcc_csm1_1' AND year='2020-2039'"
).show(8)

+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|     year|month|     model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|daily_rain_max_10_years_mm|5-day_rain_sum_max_10_years_mm|5-day_rain_sum_max_25_years_mm|largest_month_rain_10_years_mm|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|2020-2039| null|bcc_csm1_1|Annual Anomaly| FRA|         rcp26|                   1.51429|                   1.26548|                       2.44999|                       2.95582|                       -4.7716|
+---------+-----+----------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+--------

In [ ]:
for feature in monthly_features:
    print(feature)
    spark.sql(
        "SELECT DISTINCT statistics FROM future WHERE `" + feature + "` IS NOT NULL"
    ).show(3)

In [48]:
spark.sql(
    "SELECT * FROM yearly_table WHERE year='2020-2039' AND statistics='Annual Anomaly'"
).show(3)

+---------+-----+--------------------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|     year|month|               model|    statistics|ISO3|projection_rcp|daily_rain_max_25_years_mm|daily_rain_max_10_years_mm|5-day_rain_sum_max_10_years_mm|5-day_rain_sum_max_25_years_mm|largest_month_rain_10_years_mm|
+---------+-----+--------------------+--------------+----+--------------+--------------------------+--------------------------+------------------------------+------------------------------+------------------------------+
|2020-2039| null|Ensemble (10th Pe...|Annual Anomaly| DMA|         rcp60|                   -25.177|                   -17.202|                        -46.34|                       -64.286|                       -124.63|
|2020-2039| null|Ensemble (10th Pe...|Annual Anomaly| ISL|         rcp45|                   -6.9602|                

In [36]:
df_yearly.toPandas().ISO3.value_counts()

OMN    392
IDN    392
BRB    392
CHE    392
EST    392
      ... 
FRA    392
MOZ    392
TCD    392
PER    392
ERI    311
Name: ISO3, Length: 195, dtype: int64

In [49]:
sc.stop()